# Вариант поиска региона с регулярными выражениями

In [ ]:
pattern = re.compile(
    r"""
    ады|алтай|баш|Бур|Даг|Инг|КБР|Калмык|КЧР|Карел|Коми|Крым|Марий|Мордов|Саха|Сев\.? Осет|Татар|Тыва|Удмурт|Хакаси|Чеч|Чуваш|
    Алт\.? край|Кр\.? край|Крс\.? край|Перм\.? край|Прим\.? край|Став\.? край|Хаб\.? край|Заб\.? край|Кам\.? край|
    Амур|Арх|Астр|Белг|Брян|Влад|Волг|Волог|Ворон|Иван|Ирк|Кал|Калуж|Кем|Киров|Костр|Кург|Кур|Лен|Лип|Маг|Моск\.? обл|Мурм|Ниж|Новг|Новосиб|Омск|Орен|Орлов|Пенз|Пск|Рост|Ряз|Сам|Сарат|Сах|Сверд|Смол|Тамб|Твер|Том|Тул|Тюм|Ульян|Чел|Яросл|
    Москва|Санкт\.?|СПб|Севастополь|
    Евр|Ненец|ХМАО|Чукот|ЯНАО
    """,
    re.VERBOSE,
)

In [ ]:
# Пример использования
names = [
    "цфксиз василеостровского района санкт-петербург",
    "цфксиз московского района санкт-петербург",
    "цфксиз красногвардейского района санкт-петербург",
]

for name in names:
    match = pattern.search(name)
    if match:
        print(f"Регион найден: {match.group(0)} в названии '{name}'")

# Поиск региона с использованием ЛогРег

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Пример данных
data = {
    "name": [
        "цфксиз василеостровского района санкт петербург",
        "цфксиз московского района санкт петербург",
        "цфксиз красногвардейского района санкт петербург",
    ],
    "region": ["санкт петербург", "санкт петербург", "санкт петербург"],
}

df = pd.DataFrame(data)

# Векторизация текстов
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["name"])
y = df["region"]

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Обучение модели
model = LogisticRegression()
model.fit(X_train, y_train)

# Оценка модели
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Пример предсказания региона для нового названия школы
new_name = ["новая школа московского района"]
new_X = vectorizer.transform(new_name)
predicted_region = model.predict(new_X)
print(f"Предсказанный регион: {predicted_region[0]}")

# Использование spaCy для NER

In [19]:
import spacy

# Загрузка русскоязычной модели spaCy
nlp = spacy.load("ru_core_news_sm")

# Пример названий школ
school_names = [
    "цфксиз санкт-петербург",
    "цфксиз московского района санкт петербург",
    "цфксиз башкортостан",
]

# Обработка текстов
for name in school_names:
    doc = nlp(name)
    regions = [ent.text for ent in doc.ents if ent.label_ == "LOC"]
    print(f"Название школы: {name}")
    print(f"Выявленные регионы: {regions}")
    print()

Название школы: цфксиз санкт-петербург
Выявленные регионы: ['санкт-петербург']

Название школы: цфксиз московского района санкт петербург
Выявленные регионы: ['московского района', 'санкт петербург']

Название школы: цфксиз башкортостан
Выявленные регионы: []



# Использование spaCy для кастомной модели

In [20]:
import spacy
from spacy.tokens import DocBin
import random

TRAIN_DATA = [
    (
        "цфксиз василеостровского района санкт петербург",
        {"entities": [(9, 35, "REGION"), (36, 51, "CITY")]},
    ),
    (
        "цфксиз московского района санкт петербург",
        {"entities": [(9, 28, "REGION"), (29, 44, "CITY")]},
    ),
    (
        "цфксиз красногвардейского района санкт петербург",
        {"entities": [(9, 34, "REGION"), (35, 50, "CITY")]},
    ),
]


def create_training_data():
    db = DocBin()
    nlp = spacy.blank("ru")
    for text, annotations in TRAIN_DATA:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annotations.get("entities"):
            span = doc.char_span(start, end, label=label)
            if span is None:
                continue
            ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk("./train.spacy")


create_training_data()

In [53]:
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
import random

# Пример данных для обучения
TRAIN_DATA = [
    (
        "цфксиз василеостровского района санкт петербург",
        {"entities": [(7, 31, "DISTRICT"), (32, 47, "CITY")]},
    ),
    (
        "цфксиз московского района санкт петербург",
        {"entities": [(7, 25, "DISTRICT"), (26, 41, "CITY")]},
    ),
    (
        "цфксиз красногвардейского района санкт петербург",
        {"entities": [(7, 32, "DISTRICT"), (33, 48, "CITY")]},
    ),
    # Добавьте больше примеров для лучшего обучения
]

# Создаем пустую модель для русского языка
nlp = spacy.blank("ru")

# Создаем NER компонент
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")
else:
    ner = nlp.get_pipe("ner")

# Добавляем метки в NER
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Обучение модели
nlp.begin_training()

# Обучение модели
for itn in range(100):  # Количество эпох
    random.shuffle(TRAIN_DATA)
    losses = {}
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        for text, annotations in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], drop=0.5, losses=losses)
    print(f"Losses at iteration {itn}: {losses}")

# Сохраняем обученную модель
nlp.to_disk("./custom_ner_model")

Losses at iteration 0: {'ner': 11.985911577939987}
Losses at iteration 1: {'ner': 12.798401474952698}
Losses at iteration 2: {'ner': 12.107979238033295}
Losses at iteration 3: {'ner': 11.239227592945099}
Losses at iteration 4: {'ner': 10.391950190067291}
Losses at iteration 5: {'ner': 9.15412774682045}
Losses at iteration 6: {'ner': 8.029295608401299}
Losses at iteration 7: {'ner': 7.55447481200099}
Losses at iteration 8: {'ner': 7.068738471716642}
Losses at iteration 9: {'ner': 8.016470536589622}
Losses at iteration 10: {'ner': 7.17597384378314}
Losses at iteration 11: {'ner': 7.318301312625408}
Losses at iteration 12: {'ner': 9.946942746639252}
Losses at iteration 13: {'ner': 11.74552506674081}
Losses at iteration 14: {'ner': 11.978147700428963}
Losses at iteration 15: {'ner': 7.62208810262382}
Losses at iteration 16: {'ner': 5.833457122091204}
Losses at iteration 17: {'ner': 4.843606917653233}
Losses at iteration 18: {'ner': 3.873338021337986}
Losses at iteration 19: {'ner': 2.33129

In [54]:
import spacy

# Загрузка обученной модели
nlp = spacy.load("./custom_ner_model")

# Пример названий школ
school_names = [
    "санкт петербург школа № 1 василеостровский район",
    "сшор московского района санкт петербург",
    "цфксиз школа красногвардейского района санкт петербург",
]

# Обработка текстов
for name in school_names:
    doc = nlp(name)
    print(f"Название школы: {name}")
    for ent in doc.ents:
        print(f"{ent.text}: {ent.label_}")
    print()

Название школы: санкт петербург школа № 1 василеостровский район
санкт петербург: CITY
№ 1: CITY
василеостровский район: CITY

Название школы: сшор московского района санкт петербург
московского района: DISTRICT
санкт петербург: CITY

Название школы: цфксиз школа красногвардейского района санкт петербург
школа красногвардейского: DISTRICT
санкт петербург: CITY



## Использование предобученной модели SpaCy

In [55]:
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
import random

# Пример данных для обучения
TRAIN_DATA = [
    (
        "цфксиз василеостровского района санкт петербург",
        {"entities": [(7, 31, "DISTRICT"), (32, 47, "CITY")]},
    ),
    (
        "цфксиз московского района санкт петербург",
        {"entities": [(7, 25, "DISTRICT"), (26, 41, "CITY")]},
    ),
    (
        "цфксиз красногвардейского района санкт петербург",
        {"entities": [(7, 32, "DISTRICT"), (33, 48, "CITY")]},
    ),
    (
        "санкт петербург школа № 1 василеостровский район",
        {"entities": [(0, 15, "CITY"), (26, 48, "DISTRICT")]},
    ),
    (
        "сшор московского района санкт петербург",
        {"entities": [(5, 23, "DISTRICT"), (24, 39, "CITY")]},
    ),
    (
        "цфксиз школа красногвардейского района санкт петербург",
        {"entities": [(13, 38, "DISTRICT"), (39, 54, "CITY")]},
    ),
    # Добавьте больше примеров для лучшего обучения
]

# Создаем пустую модель для русского языка
nlp = spacy.load("ru_core_news_sm")

# Получение NER компонента
ner = nlp.get_pipe("ner")

# Добавляем метки в NER
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Обучение модели
optimizer = nlp.resume_training()

# Обучение модели
for itn in range(100):  # Количество эпох
    random.shuffle(TRAIN_DATA)
    losses = {}
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        for text, annotations in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], drop=0.5, losses=losses)
    print(f"Losses at iteration {itn}: {losses}")

# Сохраняем обученную модель
nlp.to_disk("./custom_ner_model")

Losses at iteration 0: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 22.797805700156058}
Losses at iteration 1: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 21.65801942284679}
Losses at iteration 2: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 16.219577122910437}
Losses at iteration 3: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 15.489335578696682}
Losses at iteration 4: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 14.59880506159243}
Losses at iteration 5: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 25.825605699094012}
Losses at iteration 6: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 22.36062752874568}
Losses at iteration 7: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 23.188023556023836}
Losses at iteration 8: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 14.0267062576022}
Losses at iteration 9: {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser

In [58]:
import spacy

# Загрузка обученной модели
nlp = spacy.load("./custom_ner_model")

# Пример названий школ
school_names = [
    "санкт петербург школа № 1 выборгский район",
    "сшор московского района санкт петербург",
    "цфксиз школа № 4 санкт петербурга",
    "сшор московского района санкт петербург",
]

# Обработка текстов
for name in school_names:
    doc = nlp(name)
    print(f"Название школы: {name}")
    for ent in doc.ents:
        print(f"{ent.text}: {ent.label_}")
    print()

Название школы: санкт петербург школа № 1 выборгский район
санкт петербург: CITY
выборгский район: DISTRICT

Название школы: сшор московского района санкт петербург
московского района: DISTRICT
санкт петербург: CITY

Название школы: цфксиз школа № 4 санкт петербурга
санкт петербурга: CITY

Название школы: сшор московского района санкт петербург
московского района: DISTRICT
санкт петербург: CITY



# Использование Hugging Face Transformers для NER (не пошло)

In [ ]:
from transformers import pipeline

# Убедитесь, что вы установили PyTorch или TensorFlow
import torch

print(torch.__version__)  # Убедитесь, что PyTorch установлен

# Загрузка модели для NER
ner_pipeline = pipeline(
    "ner",
    model="xlm-roberta-large-finetuned-conll03-english",
    tokenizer="xlm-roberta-large-finetuned-conll03-english",
)

# Пример названий школ
school_names = [
    "цфксиз василеостровского района санкт петербург",
    "цфксиз московского района санкт петербург",
    "цфксиз красногвардейского района санкт петербург",
]

# Обработка текстов
for name in school_names:
    ner_results = ner_pipeline(name)
    regions = [entity["word"] for entity in ner_results if entity["entity"] == "LOC"]
    print(f"Название школы: {name}")
    print(f"Выявленные регионы: {regions}")
    print()

# SIMPLE модель

In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score


In [18]:
# Пример данных
X_data = {"X": ["School A", "School B", "School J", "School D", "School E"]}
reference_data = {
    "id": [1, 2, 3, 4, 5, 6],
    "reference": [
        "School A",
        "School B",
        "School C",
        "School D",
        "School F",
        "School G",
    ],
}
y_data = {"id": [1, 2, 3, 4, 6]}

X_valid = pd.DataFrame(X_data)
reference = pd.DataFrame(reference_data)
y_valid = pd.DataFrame(y_data)


In [19]:
X_valid.head()

,X
0,School A
1,School B
2,School J
3,School D
4,School E


In [20]:
y_valid.head()

,id
0,1
1,2
2,3
3,4
4,6


In [21]:
reference.head()

,id,reference
0,1,School A
1,2,School B
2,3,School C
3,4,School D
4,5,School F


In [22]:
def find_matches(X, reference, top_k=5):
    Y_pred = []
    for name in X["X"]:
        matches = reference[
            reference["reference"].str.contains(name.split()[-1], na=False)
        ]
        top_matches = matches.head(top_k)["id"].tolist()
        if len(top_matches) < top_k:
            top_matches += [None] * (
                top_k - len(top_matches)
            )  # Pad with None if less than top_k matches
        Y_pred.append(top_matches)
    return Y_pred


# Получение предсказаний
y_pred = find_matches(X_valid, reference)

print(y_pred)


[[1, None, None, None, None], [2, None, None, None, None], [None, None, None, None, None], [4, None, None, None, None], [None, None, None, None, None]]


In [23]:
def accuracy_top_k(Y, Y_pred, k):
    correct = 0
    for y_true, y_pred in zip(Y["id"], Y_pred):
        if y_true in y_pred[:k]:
            correct += 1
    return correct / len(Y)


# Расчет accuracy_top_k
accuracy_top_1 = accuracy_top_k(y_valid, y_pred, 1)
accuracy_top_3 = accuracy_top_k(y_valid, y_pred, 3)
accuracy_top_5 = accuracy_top_k(y_valid, y_pred, 5)

print(f"Accuracy@1: {accuracy_top_1:.2f}")
print(f"Accuracy@3: {accuracy_top_3:.2f}")
print(f"Accuracy@5: {accuracy_top_5:.2f}")


Accuracy@1: 0.60
Accuracy@3: 0.60
Accuracy@5: 0.60


In [24]:
import pandas as pd

# Пример предсказаний и истинных значений
predictions = [
    [1, None, None, None, None],
    [2, None, None, None, None],
    [None, None, None, None, None],
    [4, None, None, None, None],
    [None, None, None, None, None],
]
true_values = [1, 2, 3, 4, 6]


def check_top_1(predictions, true_values):
    result = []
    for pred, true in zip(predictions, true_values):
        if pred[0] == true:
            result.append(False)
        else:
            result.append(True)
    return result


# Получение результата
result = check_top_1(predictions, true_values)
print(result)


[False, False, True, False, True]


In [25]:
import numpy as np

# Пример данных
X_data = np.array(["School A", "School B", "School J", "School D", "School E"])
reference_data = np.array(
    [
        [1, "School A"],
        [2, "School B"],
        [3, "School C"],
        [4, "School D"],
        [5, "School F"],
        [6, "School G"],
    ]
)
y_data = np.array([1, 2, 3, 4, 6])


def find_matches(X, reference, top_k=5):
    Y_pred = []
    for name in X:
        matches = reference[np.char.find(reference[:, 1], name.split()[-1]) >= 0]
        top_matches = matches[:top_k, 0].astype(int).tolist()
        if len(top_matches) < top_k:
            top_matches += [None] * (
                top_k - len(top_matches)
            )  # Pad with None if less than top_k matches
        Y_pred.append(top_matches)
    return Y_pred


# Получение предсказаний
y_pred = find_matches(X_data, reference_data)

print(y_pred)


def accuracy_top_k(Y, Y_pred, k):
    correct = 0
    for y_true, y_pred in zip(Y, Y_pred):
        if y_true in y_pred[:k]:
            correct += 1
    return correct / len(Y)


# Расчет accuracy_top_k
accuracy_top_1 = accuracy_top_k(y_data, y_pred, 1)
accuracy_top_3 = accuracy_top_k(y_data, y_pred, 3)
accuracy_top_5 = accuracy_top_k(y_data, y_pred, 5)

print(f"Accuracy@1: {accuracy_top_1:.2f}")
print(f"Accuracy@3: {accuracy_top_3:.2f}")
print(f"Accuracy@5: {accuracy_top_5:.2f}")

# Пример предсказаний и истинных значений
predictions = np.array(
    [
        [1, None, None, None, None],
        [2, None, None, None, None],
        [None, None, None, None, None],
        [4, None, None, None, None],
        [None, None, None, None, None],
    ]
)
true_values = np.array([1, 2, 3, 4, 6])


def check_top_1(predictions, true_values):
    result = []
    for pred, true in zip(predictions, true_values):
        if pred[0] == true:
            result.append(False)
        else:
            result.append(True)
    return result


# Получение результата
result = check_top_1(predictions, true_values)
print(result)


[[1, None, None, None, None], [2, None, None, None, None], [None, None, None, None, None], [4, None, None, None, None], [None, None, None, None, None]]
Accuracy@1: 0.60
Accuracy@3: 0.60
Accuracy@5: 0.60
[False, False, True, False, True]


In [26]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Пример данных
X_data = np.array(["School A", "School B", "School J", "School D", "School E"])
reference_data = np.array(
    [
        [1, "School A"],
        [2, "School B"],
        [3, "School C"],
        [4, "School D"],
        [5, "School F"],
        [6, "School G"],
    ]
)
y_data = np.array([1, 2, 3, 4, 6])

# Векторизация текстов
vectorizer = TfidfVectorizer().fit(np.append(X_data, reference_data[:, 1]))
X_vec = vectorizer.transform(X_data)
reference_vec = vectorizer.transform(reference_data[:, 1])


def find_matches(X_vec, reference_vec, reference_ids, top_k=5):
    Y_pred = []
    for x in X_vec:
        similarities = cosine_similarity(x, reference_vec).flatten()
        top_indices = similarities.argsort()[-top_k:][::-1]
        top_matches = [(reference_ids[i], similarities[i]) for i in top_indices]
        if len(top_matches) < top_k:
            top_matches += [(None, 0.0)] * (
                top_k - len(top_matches)
            )  # Pad with (None, 0.0) if less than top_k matches
        Y_pred.append(top_matches)
    return Y_pred


# Получение предсказаний
y_pred = find_matches(X_vec, reference_vec, reference_data[:, 0].astype(int))

print(y_pred)


def accuracy_top_k(Y, Y_pred, k):
    correct = 0
    for y_true, y_pred in zip(Y, Y_pred):
        top_k_ids = [match[0] for match in y_pred[:k]]
        if y_true in top_k_ids:
            correct += 1
    return correct / len(Y)


# Расчет accuracy_top_k
accuracy_top_1 = accuracy_top_k(y_data, y_pred, 1)
accuracy_top_3 = accuracy_top_k(y_data, y_pred, 3)
accuracy_top_5 = accuracy_top_k(y_data, y_pred, 5)

print(f"Accuracy@1: {accuracy_top_1:.2f}")
print(f"Accuracy@3: {accuracy_top_3:.2f}")
print(f"Accuracy@5: {accuracy_top_5:.2f}")

# Пример предсказаний и истинных значений
predictions = [
    [(1, 1.0), (None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0)],
    [(2, 1.0), (None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0)],
    [(None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0)],
    [(4, 1.0), (None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0)],
    [(None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0)],
]
true_values = np.array([1, 2, 3, 4, 6])


def check_top_1(predictions, true_values):
    result = []
    for pred, true in zip(predictions, true_values):
        if pred[0][0] == true:
            result.append(False)
        else:
            result.append(True)
    return result


# Получение результата
result = check_top_1(predictions, true_values)
print(result)


[[(6, 1.0), (5, 1.0), (4, 1.0), (3, 1.0), (2, 1.0)], [(6, 1.0), (5, 1.0), (4, 1.0), (3, 1.0), (2, 1.0)], [(6, 1.0), (5, 1.0), (4, 1.0), (3, 1.0), (2, 1.0)], [(6, 1.0), (5, 1.0), (4, 1.0), (3, 1.0), (2, 1.0)], [(6, 1.0), (5, 1.0), (4, 1.0), (3, 1.0), (2, 1.0)]]
Accuracy@1: 0.20
Accuracy@3: 0.40
Accuracy@5: 0.80
[False, False, True, False, True]


In [27]:
import numpy as np

# Пример данных
X_data = np.array(["School A", "School B", "School J", "School D", "School E"])
reference_data = np.array(
    [
        [1, "School A"],
        [2, "School B"],
        [3, "School C"],
        [4, "School D"],
        [5, "School F"],
        [6, "School G"],
    ]
)
y_data = np.array([1, 2, 3, 4, 6])


def find_matches(X, reference, top_k=5):
    Y_pred = []
    for name in X:
        matches = [(ref_id, 1.0) for ref_id, ref_name in reference if name == ref_name]
        top_matches = matches[:top_k]
        if len(top_matches) < top_k:
            top_matches += [(None, 0.0)] * (
                top_k - len(top_matches)
            )  # Pad with (None, 0.0) if less than top_k matches
        Y_pred.append(top_matches)
    return Y_pred


# Получение предсказаний
y_pred = find_matches(X_data, reference_data)

print(y_pred)


def accuracy_top_k(Y, Y_pred, k):
    correct = 0
    for y_true, y_pred in zip(Y, Y_pred):
        top_k_ids = [match[0] for match in y_pred[:k]]
        if y_true in top_k_ids:
            correct += 1
    return correct / len(Y)


# Расчет accuracy_top_k
accuracy_top_1 = accuracy_top_k(y_data, y_pred, 1)
accuracy_top_3 = accuracy_top_k(y_data, y_pred, 3)
accuracy_top_5 = accuracy_top_k(y_data, y_pred, 5)

print(f"Accuracy@1: {accuracy_top_1:.2f}")
print(f"Accuracy@3: {accuracy_top_3:.2f}")
print(f"Accuracy@5: {accuracy_top_5:.2f}")

# Пример предсказаний и истинных значений
predictions = [
    [(1, 1.0), (None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0)],
    [(2, 1.0), (None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0)],
    [(None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0)],
    [(4, 1.0), (None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0)],
    [(None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0)],
]
true_values = np.array([1, 2, 3, 4, 6])


def check_top_1(predictions, true_values):
    result = []
    for pred, true in zip(predictions, true_values):
        if pred[0][0] == true:
            result.append(False)
        else:
            result.append(True)
    return result


# Получение результата
result = check_top_1(predictions, true_values)
print(result)


[[('1', 1.0), (None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0)], [('2', 1.0), (None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0)], [(None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0)], [('4', 1.0), (None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0)], [(None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0), (None, 0.0)]]
Accuracy@1: 0.00
Accuracy@3: 0.00
Accuracy@5: 0.00
[False, False, True, False, True]


# Данные для тестирования

In [28]:
x = np.array(["School A", "School B", "School J", "School D", "School E"])
y = np.array([10, 20, 30, 40, 60])
reference_id = np.array([10, 20, 30, 40, 50, 60])
reference_name = np.array(
    ["School A", "School B", "New School AB", "School D", "School F", "School G"]
)


# Рализация на fuzzywuzzy

In [29]:
import numpy as np
from fuzzywuzzy import process


def find_matches(X, reference_id, reference_name, top_k=5):
    y_pred = []
    for name in X:
        matches = process.extract(name, reference_name, limit=top_k)
        top_matches = [
            (reference_id[reference_name.index(match[0])], match[1] / 100.0)
            for match in matches
        ]
        if len(top_matches) < top_k:
            top_matches += [(None, 0.0)] * (top_k - len(top_matches))
        y_pred.append(top_matches)
    return y_pred


# Пример данных
X_data = np.array(["School A", "School B", "School J", "School D", "School E"])
reference_id = np.array([1, 2, 3, 4, 5, 6])
reference_name = np.array(
    ["School A", "School B", "School C", "School D", "School F", "School G"]
)

# Получение предсказаний
y_pred = find_matches(X_data, reference_id, reference_name)

print(y_pred)

# Пример данных для проверки
y_true = np.array([1, 2, 3, 4, 6])


# Функция accuracy_top_k для проверки
def accuracy_top_k(y_true, y_pred, k):
    correct = 0
    for true_id, pred_list in zip(y_true, y_pred):
        top_k_ids = [pred[0] for pred in pred_list[:k]]
        if true_id in top_k_ids:
            correct += 1
    return correct / len(y_true)


# Расчет accuracy_top_k
accuracy_top_1 = accuracy_top_k(y_true, y_pred, 1)
accuracy_top_3 = accuracy_top_k(y_true, y_pred, 3)
accuracy_top_5 = accuracy_top_k(y_true, y_pred, 5)

print(f"Accuracy@1: {accuracy_top_1:.2f}")
print(f"Accuracy@3: {accuracy_top_3:.2f}")
print(f"Accuracy@5: {accuracy_top_5:.2f}")


# Функция check_top_1 для проверки
def check_top_1(predictions, true_values):
    result = []
    for pred_list, true in zip(predictions, true_values):
        if pred_list[0][0] == true:
            result.append(False)
        else:
            result.append(True)
    return result


# Получение результата
result = check_top_1(y_pred, y_true)
print(result)


AttributeError: 'numpy.ndarray' object has no attribute 'index'

# Первая версия accuracy_top_k по всем предсказаниям (включая None)

In [ ]:
def accuracy_top_k(true_values, predictions, k):
    correct = 0
    for true_id, pred_list in zip(true_values, predictions):
        top_k_ids = [pred[0] for pred in pred_list[:k]]
        if true_id in top_k_ids:
            correct += 1
    return correct / len(true_values)


# Функция для нахождения аббревиатур

In [ ]:
# Функция для нахождения аббревиатур
def find_abbreviations(name):
    abbreviations = re.findall(r"\b[А-ЯЁ]+[а-яё]*+[А-ЯЁ]+\b", name)
    return abbreviations


# Применяем функцию ко всем строкам столбца name
reference["abbreviations"] = reference["name"].apply(find_abbreviations)
abbreviations_reference = (
    reference["abbreviations"].explode().dropna().unique().tolist()
)


# полный словарь аббревиатур

In [ ]:
# Создаем полный словарь аббревиатур
abbreviation_dict = {
    "ДИНАМО": "динамо",  # Название
    "ИСТОРИЯ": "история",  # Название
    "ОЛИМПИЯ": "олимпия",  # Название
    "СИЯНИЕ": "сияние",  # Название
    "ПГУФКСиТ": "поволжский государственный университет физической культуры спорта и туризма",  # Название
    "СиТ": "спорта и туризма",  # Название
    "ЛЕДОВАЯ": "ледовая",  # Название
    "ШФКнК": "школа фигурного катания на коньках",  # Тип организации в названии
    "ЦПФКиХ": "центр подготовки по фигурному катанию на коньках и хоккею",  # Тип организации в названии
    "ВОСТОЧНАЯ": "восточная",  # Название
    "ФиЗ": "физкультуры и здоровья",  # Название
    "НАДЕЖДЫ": "надежды",  # Название
    "ФККиХ": "фигурному катанию на коньках и хоккею",  # Тип организации в названии
    "ЦФиЗ": "центр физкультуры и здоровья",  # Название
    "АГО": "ангарский городской округ",  # Регион
    "АЛВС": "академия ледовых видов спорта",  # Тип школы
    "АНО": "автономная некоммерческая организация",  # ОПФ
    "АНФСО": "автономная некоммерческая физкультурно-спортивная организация",  # ОПФ
    "АО": "автономный округ",  # Регион
    "АУ": "автономное учреждение",  # ОПФ
    "АФК": "академия фигурного катания",  # Тип школы
    "БарС": "березники арена спорт",  # Название
    "БУ": "бюджетное учреждение",  # ОПФ
    "ВДВ": "воздушно-десантные войска",  # Часть названия, принадлежность
    "ВИТЯЗЬ": "витязь",  # Название
    "ВО": "василеостровского района",  # Район
    "ВСШОР": "всеволжская спортивная школа олимпийского резерва",  # Школа # Город
    "ВФСО": "всероссийское физкультурно-спортивное общество",  # ОПФ
    "ГАУ": "государственное автономное учреждение",  # ОПФ
    "ГБОУ": "государственное бюджетное образовательное учреждение",
    "ГБПОУ": "государственное бюджетное профессиональное образовательное учреждение",  # ОПФ
    "ГБУ": "государственное бюджетное учреждение",  # ОПФ
    "ГЛАЙД": "глайд название спортивного клуба",  # Название
    "ГОАУ": "государственное областное автономное учреждение",  # Уточнить последовательность слов для унификации
    "ГУ": "государственное учреждение",  # ОПФ
    "ДО": "дополнительного образования",  # ОПФ
    "ДОД": "дополнительного образования детей",  # Вид деятельности
    "ДС": "дворец спорта",  # Тип учреждения
    "ДЮСОШ": "детско-юношеская спортивно-оздоровительная школа",  # Тип учреждения
    "ДЮСШ": "детско-юношеская спортивная школа",  # Тип учреждения
    "ДЮЦ": "детско-юношеский центр",  # Тип учреждения
    "ЗВС": "зимние виды спорта",  # Тип учреждения
    "ЗОЛОТЫЕ": "золотые",  # Название
    "ИО": "иркутская область",  # Регион
    "ИП": "индивидуальный предприниматель",  # ОПФ
    "ИРО": "иркутское региональное отделение",  # Тип организации (уникальный)
    "КГБУ": "краевое государственное бюджетное учреждение",  # ОПФ
    "КК": "краснодарского края",  # Регион
    "КО": [
        "калужская область",
        "костромская область",
        "кировская область",
        "калининградская область",
    ],  # Необходимо учесть различные регионы. Поиск осуществлять сразу в нескольких.
    "КОГАУ": "кировское областное государственное автономное учреждение",  # ОПФ
    "КРОО": "краевая региональная общественная организация",  # ОПФ
    "КФК": "клуб фигурного катания",  # Тип учреждения
    "КФКнК": "клуб фигурного катания на коньках",  # Тип учреждения
    "ЛВС": "ледовые виды спорта",  # Тип учреждения
    "ЛД": "ледовый дворец",  # Тип учреждения
    "ЛДС": "ледовый дворец спорта",  # Тип учреждения
    "ЛО": "ленинградская Область",  # Регион
    "ЛСВС": "ледовым видам спорта",  # Тип учреждения - ОШИБКА!, лишняя буква
    "ЛСК": "ледовый спортивный комплекс",  # Тип учреждения
    "ЛЦ": "ледовый центр",  # Тип учреждения
    "МА": "московская академия",  # Тип учреждения - ОШИБКА!, часть МАФКК
    "МАОУ": "муниципальное автономное образовательное учреждение",  # ОПФ
    "МАУ": "муниципальное автономное учреждение",  # ОПФ
    "МАУДО": "муниципальное автономное учреждение дополнительного образования",  # ОПФ
    "МАУСШ": "муниципальное автономное учреждение спортивная школа",  # ОПФ
    "МАФК": "московская академия фигурного катания на коньках",  # Тип учреждения - ОШИБКА!, часть МАФКК
    "МАФКК": "московская академия фигурного катания на коньках",  # Тип учреждения, иногда разбивается на составляющие
    "МАФСУ": "муниципальное автономное физкультурно-спортивное учреждение",  # Тип учреждения
    "МБОУ": "муниципальное бюджетное образовательное учреждение",  # ОПФ
    "МБУ": "муниципальное бюджетное учреждение",  # ОПФ
    "МБУДО": "муниципальное бюджетное учреждение дополнительного образования",  # ОПФ
    "МБУС": "муниципальное бюджетное учреждение спорта",  # ОПФ
    "МБФСУ": "муниципальное бюджетное физкультурно-спортивное учреждение",  # ОПФ
    "МДМиС": "министерство по делам молодёжи и спорту",  # ФОИВ
    "МКУ": "муниципальное казенное учреждение",  # ОПФ
    "МО": [
        "муниципального образования",
        "московской области",
        "мурманской области",
        "министерства обороны",
    ],  # Район, Регион
    "МО РФ": ["министерства обороны россии"],  # Район, Регион
    "МО РОССИИ": ["министерства обороны россии"],  # Район, Регион
    "МОСГОРСПОРТ": "московская дирекция по развитию массового спорта",  # Наименование организации
    "МОУ": "муниципальное образовательное учреждение",  # ОПФ
    "МУ": "муниципальное учреждение",  # ОПФ
    "НАШИ": "наши название спортивного клуба",  # Название
    "НАО": "ненецкий автономный округ",  # Регион
    "НЛФК": "национальная лига фигурного катания",  # Название
    "НО": "нижегородская область",  # Регион
    "НП": "некоммерческое партнерство",  # ОПФ
    "ОГАУ": "областное государственное автономное учреждение",  # ОПФ
    "ОГО": "общественно-государственное объединение",  # ОПФ
    "ОКСШОР": "областная комплексная спортивная школа олимпийского резерва",  # ОПФ
    "ОЛИМП": "олимп название спортивного клуба",  # Название
    "ОО": ["общественная организация", "общественное объединение"],  # ОПФ
    "ООО": "общество с ограниченной ответственностью",  # ОПФ
    "ООФФК": "общественная организация федерация фигурного катания",  # Название
    "ОРК": "олимпийского резерва, комплексная",  # Выпендрежники-бюрократы
    "ОУР": "училище (техникум) олимпийского резерва",  # Название
    "ПБМУ": "первоуральское муниципальное бюджетное учреждение",  # ОПФ с регионом (город) с ошибкой, смотри ПБМУ
    "ПМБУ": "первоуральское муниципальное бюджетное учреждение",  # ОПФ с регионом (город)
    "ПО": "псковской области",  # Регион
    "ПРОО": "псковская региональная общественная организация",  # ОПФ с регионом (город)
    "РБ": ["республика башкортостан", "республика беларусь"],
    "РК": [
        "республика карелия",
        "республика калмыкия",
        "республика коми",
        "республика крым",
        "республика казахстан",
    ],  # Регион
    "РМ": "республика мордовия",  # Регион
    "РМЭ": "республика марий эл",  # Регион
    "РО": [
        "рязанская область",
        "региональное объединение",
        "ростовская область",
    ],  # ОПФ, регион
    "РОО": "региональная общественная организация",  # ОПФ
    "РОФФКК": "ростовская областная федерация фигурного катания на коньках",  # ОПФ, регион
    "РС": "республика саха якутия",  # Регион
    "РСОО": "региональная спортивная общественная организация",  # ОПФ
    "РССШ": "республики саха якутия спортивная школа",  # ОПФ с лишними буквами
    "РСШОР": "республиканская спортивная школа олимпийского резерва",  # Тип учреждения
    "РСЯ": "республика саха якутия",  # Регион
    "РС (Я)": "республика саха якутия",  # Регион
    "РС Я": "республика саха якутия",  # Регион
    "РТ": "республика татарстан",  # Регион
    "РФ": "российская федерация",  # Регион
    "РФСОО": "региональная физкультурно спортивная общественная организация",  # ОПФ
    "РЦСП": "региональный центр спортивной подготовки",  # Тип организации - Региональный центр спортивной подготовки сборных команд
    "РЦСПЗВС": "региональный центр спортивной подготовки по зимним видам спорта",  # Тип организации
    "САХА": "республики саха",  # Регион
    "СБС": "сбс название спортивного клуба",  # Название
    "СК": "спортивный клуб",  # Тип организации
    "СКА": "спортивный клуб армии",  # Тип организации
    "СКФК": "Спортивный Клуб Фигурного Катания",  # Тип организации
    "СО": [
        "самарская область",
        "свердловская область",
        "смоленская область",
    ],  # ОПФ, регион
    "СОГБУ": "смоленское областное государственное бюджетное учреждение",  # ОПФ, регион
    "СОГБОУ": "смоленское областное государственное бюджетное образовательное учреждение",  # ОПФ, регион
    "СПБ": "санкт петербург",  # Регион
    "СТАРТАЙС": "Стартайс название спортивного клуба",  # Название
    "СТЦ": "спортивно тренировочный центр",  # Тип организации
    "СФК": "секция фигурного катания",  # Тип организации
    "СФФК": "спортивная федерация фигурного катания на коньках",  # ОПФ
    "СШ": "спортивная школа",  # Тип организации
    "СШОР": "спортивная школа олимпийского резерва",  # Тип организации
    "СШФКК": "спортивная школа фигурного катания ",  # Тип организации
    "ТИМ": "тим название спортивного клуба",  # Название
    "ТИТУЛ": "титул название спортивного клуба",  # Название
    "ТО": "тульской области",  # Регион
    "ТОП": "топ название спортивного клуба",  # Название
    "УЗОР": "узор название спортивного клуба",  # Название
    "УОР": "училище олимпийского резерва",  # Название
    "УР": "удмуртской республики",  # Регион
    "УФФК": "удмуртская федерация фигурного катания",  # ОПФ, регион
    "ФАУ": "федеральное автономное учреждение",  # ОПФ
    "ФК": "фигурного катания",  # Тип учреждения
    "ФКиС": "физической культуры и спорта",  # Тип учреждения, уникальный
    "ФКиШТ": "фигурному катанию на коньках и шорт треку",  # Тип учреждения в названии
    "ФККиШТ": "фигурному катанию на коньках и шорт треку",  # Тип учреждения в названии
    "ФКК": "фигурного катания на коньках",  # Тип учреждения
    "ФКСиЗ": "физической культуры спорта и здоровья",  # Тип учреждения
    "ФОК": "физкультурно оздоровительный комплекс",  # Тип учреждения
    "ФС": "физкультурно спортивное",  # Тип учреждения
    "ФСК": "физкультурно спортивный комплекс",  # Тип учреждения
    "ФСО": "физкультурно спортивное объединение",  # Тип учреждения
    "ФФК": "федерация фигурного катания",  # Тип учреждения
    "ФФКК": "федерация фигурного катания на коньках",  # Тип учреждения
    "ФФККВО": "федерация фигурного катания на коньках владимирской области",  # Тип учреждения и регион, можно как-то выделять
    "ФФККРК": "федерация фигурного катания на коньках республики коми",  # Тип учреждения и регион, можно как-то выделять
    "ФФККСК": "федерация фигурного катания на коньках ставропольского края",  # Тип учреждения и регион, можно как-то выделять
    "ФФКПО": "федерация фигурного катания на коньках псковской области",  # Тип учреждения и регион, можно как-то выделять
    "ХК": "хоккейный клуб",  # Наименование
    "ХМАО": "ханты мансийский автономный округ",  # Регион
    "ХШФК": "спортивный клую по хоккею с шайбой и фигурному катанию",  # РЕГИОНАЛЬНАЯ ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ "СПОРТИВНЫЙ КЛУБ ПО ХОККЕЮ С ШАЙБОЙ И ФИГУРНОМУ КАТАНИЮ"
    "ЦЗВС": "центр зимних видов спорта",  # Тип учреждения
    "ЦОП": "центр олимпийской подготовки",  # Тип учреждения
    "ЦпЛВС": "центр по ледовым видам спорта",  # Тип учреждения в названии
    "ЦПСР": "центр подготовки спортивного резерва",  # Тип учреждения
    "ЦРС": "центр развития спорта",  # Тип учреждения в названии
    "ЦРФК": "центр развития фигурного катания",  # Тип учреждения в названии
    "ЦСКА": "центральный спортивный клуб армии",  # Название
    "ЦСП": "центр спортивной подготовки",  # Тип учреждения в названии
    "ЦФК": "центр физической культуры",  # Тип учреждения в названии
    "ЦФКСиЗ": "центр физической культуры спорта и здоровья",  # Тип учреждения в названии
    "ЦФКиЗ": "центр физической культуры и здоровья",  # Тип учреждения в названии
    "ЦФКиС": "центр физической культуры и спорта",  # Тип учреждения в названии
    "ЧОУ": "частное образовательное учреждение",  # ОПФ
    "ЧР": ["чувашская республика", "чеченская республика"],  # Регион
    "ЧУ": "частное учреждение",  # ОПФ
    "ЧУДО": "частное учреждение дополнительного образования",  # ОПФ
    "ШФК": "школа фигурного катания",  # Тип учреждения
    "ЮНАРМЕЙЦЫ": "юнармецны название спортивного клуба",  # Название
    "ЯНАО": "ямало ненецкий автономный округ",  # Регион
}


In [ ]:
# Функция для нахождения аббревиатур
def find_abbreviations(name):
    abbreviations = re.findall(r"\b[А-ЯЁ]+[а-яё]*+[А-ЯЁ]+\b", name)
    return abbreviations


# Применяем функцию ко всем строкам столбца name
reference["abbreviations"] = reference["name"].apply(find_abbreviations)
abbreviations_reference = (
    reference["abbreviations"].explode().dropna().unique().tolist()
)


In [ ]:
reference.head()

,id,name,region,abbreviations
0,69,Кировска,Мурманская область,[]
1,70,Клин спортивный,Московская область,[]
2,71,КО СШ по ЗВС,Калининградская область,"[КО, СШ, ЗВС]"
3,8,Айсберг,Свердловская область,[]
4,73,Команда Дмитриева,Челябинская область,[]


In [ ]:
len(abbreviations_reference)

90

Определим аббревиатуры также из train выборки

In [ ]:
train = pd.read_csv("../../data/splited/train.csv")

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       551 non-null    object
 1   school_id  551 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 8.7+ KB


In [ ]:
train.head()

,name,school_id
0,"Москва, ГБУ МАФКК школа ""Мечта""",102
1,"Калужская область, СШ ""Космос""",78
2,"МО 7-й округ, ООО ""Династия""",49
3,АНФСО СКФК Спартак,164
4,"Челябинская область, ИП Дмитриев А.А.",73


In [ ]:
# Применяем функцию ко всем строкам столбца name
train["abbreviations"] = train["name"].apply(find_abbreviations)
abbreviations_train = train["abbreviations"].explode().dropna().unique().tolist()


In [ ]:
all_abbreviations = abbreviations_reference + abbreviations_train
all_abbreviations = set(all_abbreviations)
print(len(all_abbreviations))


175


In [ ]:
abbreviation_dict_new = {}
for abbr in all_abbreviations:
    if abbr not in abbreviation_dict:
        abbreviation_dict_new[abbr] = ""


In [ ]:
abbreviation_dict_new

{'ОЛИМП': '',
 'ГЛАЙД': '',
 'ЦпЛВС': '',
 'ЦФКиС': '',
 'КФКнК': '',
 'ЦФКиЗ': '',
 'ФКСиЗ': '',
 'НАШИ': '',
 'ФКиШТ': '',
 'БарС': '',
 'ЗОЛОТЫЕ': '',
 'ВИТЯЗЬ': '',
 'ФККиШТ': '',
 'ДИНАМО': '',
 'ИСТОРИЯ': '',
 'ОЛИМПИЯ': '',
 'СИЯНИЕ': '',
 'ПГУФКСиТ': '',
 'СиТ': '',
 'ЛЕДОВАЯ': '',
 'ШФКнК': '',
 'ЦПФКиХ': '',
 'ВОСТОЧНАЯ': '',
 'ФиЗ': '',
 'НАДЕЖДЫ': '',
 'ФККиХ': '',
 'ЦФиЗ': ''}

In [ ]:
reference[reference["id"] == 79]

,id,name,region,abbreviations
10,79,Красная звезда,Санкт-Петербург,[]


In [ ]:
train[train["school_id"] == 79]

,name,school_id,abbreviations
533,ХШФК Красная звезда,79,[ХШФК]


In [ ]:
reference[reference["name"].str.contains("ФККиХ")]

,id,name,region,abbreviations
168,179,СШ по ФККиХ,Краснодарский край,"[СШ, ФККиХ]"


In [ ]:
train[train["name"].str.contains("ФККиХ")]

,name,school_id,abbreviations


In [ ]:
abbreviations_train = train["abbreviations"].explode().dropna().unique().tolist()

In [ ]:
abbreviation_dict_small = {}
for key in abbreviation_dict:
    new_key = key.lower()
    abbreviation_dict_small[new_key] = abbreviation_dict[key]


In [ ]:
abbreviation_dict_small

{'динамо': 'динамо',
 'история': 'история',
 'олимпия': 'олимпия',
 'сияние': 'сияние',
 'пгуфксит': 'поволжский государственный университет физической культуры спорта и туризма',
 'сит': 'спорта и туризма',
 'ледовая': 'ледовая',
 'шфкнк': 'школа фигурного катания на коньках',
 'цпфких': 'центр подготовки по фигурному катанию на коньках и хоккею',
 'восточная': 'восточная',
 'физ': 'физкультуры и здоровья',
 'надежды': 'надежды',
 'фкких': 'фигурному катанию на коньках и хоккею',
 'цфиз': 'центр физкультуры и здоровья',
 'аго': 'ангарский городской округ',
 'алвс': 'академия ледовых видов спорта',
 'ано': 'автономная некоммерческая организация',
 'анфсо': 'автономная некоммерческая физкультурно-спортивная организация',
 'ао': 'автономный округ',
 'ау': 'автономное учреждение',
 'афк': 'академия фигурного катания',
 'барс': 'березники арена спорт',
 'бу': 'бюджетное учреждение',
 'вдв': 'воздушно-десантные войска',
 'витязь': 'витязь',
 'во': 'василеостровского района',
 'всшор': 'всев

## NLTK

In [ ]:
import nltk
import pymorphy3

# Загружаем необходимые ресурсы
nltk.download("punkt")
nltk.download("stopwords")

# Инициализация морфологического анализатора для русского языка
morph = pymorphy3.MorphAnalyzer()


def find_and_replace_abbreviations(name, abbreviation_dict):
    # Приводим строку к нижнему регистру
    name = name.lower()
    parts = name.split()

    possible_replacements = []
    unknown_abbrs = []

    # Регулярное выражение для поиска аббревиатур большими буквами
    uppercase_abbr_pattern = re.compile(r"\b[А-ЯЁ]{2,}\b")

    for part in parts:
        if len(part) <= 3 and part not in abbreviation_dict:
            # Пропускаем короткие слова (например, предлоги)
            possible_replacements.append([part])
        elif part in abbreviation_dict:
            replacements = abbreviation_dict[part]
            if isinstance(replacements, str):
                replacements = [replacements]
            possible_replacements.append(replacements)
        elif uppercase_abbr_pattern.match(part.upper()):
            possible_replacements.append(["unknown"])
            unknown_abbrs.append(part.upper())
        else:
            possible_replacements.append([part])

    # Генерируем все возможные комбинации
    all_combinations = list(itertools.product(*possible_replacements))

    # Формируем итоговые наименования
    final_phrases = [" ".join(combination) for combination in all_combinations]

    return {"phrases": final_phrases, "unknown": list(set(unknown_abbrs))}


# Пример использования
name = "КО ФГУП МО СШ по ЗВС и фкк"
abbr_dict = {
    "ко": ["калужская область", "костромская область"],
    "сш": ["спортивная школа", "средняя школа"],
    "звс": "зимние виды спорта",
    "фкк": "фигурному катанию на коньках",
}

result = find_and_replace_abbreviations(name, abbr_dict)
result


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\filat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\filat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'phrases': ['калужская область unknown мо спортивная школа по зимние виды спорта и фигурному катанию на коньках',
  'калужская область unknown мо средняя школа по зимние виды спорта и фигурному катанию на коньках',
  'костромская область unknown мо спортивная школа по зимние виды спорта и фигурному катанию на коньках',
  'костромская область unknown мо средняя школа по зимние виды спорта и фигурному катанию на коньках'],
 'unknown': ['ФГУП']}

# END